In [1]:
import re

import pandas as pd

## Inspect Data
See what data is available from the ClinVar file.

In [2]:
clinvar = pd.read_table("clinvar_result.txt", sep="\t")

In [3]:
list(clinvar)

['Name',
 'Gene(s)',
 'Condition(s)',
 'Clinical significance (Last reviewed)',
 'Review status',
 'GRCh37Chromosome',
 'GRCh37Location',
 'GRCh38Chromosome',
 'GRCh38Location',
 'VariationID',
 'AlleleID(s)',
 'Unnamed: 11']

In [4]:
len(clinvar)

217

In [5]:
clinvar.head()

,Name,Gene(s),Condition(s),Clinical significance (Last reviewed),Review status,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,AlleleID(s),Unnamed: 11
0,NM_152296.4(ATP1A3):c.*315G>A,ATP1A3,Dystonia 12|Alternating hemiplegia of childhood,"Likely benign(Last reviewed: Jun 14, 2016)","criteria provided, single submitter",19,42470774,19,41966622,329395,333688,NaN
1,NM_152296.4(ATP1A3):c.*298C>T,ATP1A3,Dystonia 12|Alternating hemiplegia of childhood,"Likely benign(Last reviewed: Jun 14, 2016)","criteria provided, single submitter",19,42470791,19,41966639,329396,343686,NaN
2,NM_152296.4(ATP1A3):c.*280T>A,ATP1A3,Dystonia 12|Alternating hemiplegia of childhood,"Uncertain significance(Last reviewed: Jun 14, ...","criteria provided, single submitter",19,42470809,19,41966657,329397,333691,NaN
3,NM_152296.4(ATP1A3):c.*236T>C,ATP1A3,Dystonia 12|Alternating hemiplegia of childhood,"Likely benign(Last reviewed: Jun 14, 2016)","criteria provided, single submitter",19,42470853,19,41966701,329398,349037,NaN
4,NM_152296.4(ATP1A3):c.*196_*198dupCTC,ATP1A3,Dystonia 12,"Pathogenic(Last reviewed: Jul 1, 2009)",no assertion criteria provided,19,42470891 - 42470893,19,41966739 - 41966741,12916,27955,NaN


In this file, it looks like only the variant infortation and clinical significance are relevant. Given that the position of the variant on the protein is known, the chromosome information seems redundant. Also, the review status and variation and allele ids seem irrelevant, and I don't know what to do with the condition information at this time.

__* Might be something to explore in the future.__

Also, the pathogenicity variant is in the same row as the date reviewed, and needs to be seperated.
From the variant name, I want only the position of the mutant, the ref, and the alt.

In [6]:
# Process Name
remove_db_id = lambda x: x.split(':')[1]
clinvar["variant"] = clinvar["Name"].apply(remove_db_id)

# Process Variant
is_protein_variant = lambda x: bool(x.count("p."))
clinvar["is_in_protein"] = clinvar["variant"].apply(is_protein_variant)

is_indel = lambda x: not x.count(">")
clinvar["is_indel"] = clinvar["variant"].apply(is_indel)

# Process Clinical Significance
remove_review_date = lambda x: x.split('(')[0]
clinvar["clinical_significance"] = clinvar["Clinical significance (Last reviewed)"].apply(remove_review_date)

In [7]:
print("Indel Count: %d" % len(clinvar[clinvar["is_indel"]]))
print("Non-Coding Variant Count: %d" % len(clinvar[~clinvar["is_in_protein"]]))
print("Non-Coding or Indel: %d" % len(clinvar[clinvar["is_indel"] | ~clinvar["is_in_protein"]]))

Indel Count: 10
Non-Coding Variant Count: 34
Non-Coding or Indel: 41


## Select Single Nuclotide Variants in the Protein Coding Region
I am not sure how to represent insertions and deletions for the approach I am planning to take, where I represent each variant as a one-hot vector (the format used in this paper: https://doi.org/10.1093/bioinformatics/bty211, and to be honest the only format I am aware of as of now).


For now, I will deal with this by filtering out the indels. Also, I will filter out the missense mutations as these also can't be exactly represented as a one-hot vector.

Also, I am hoping to analyze mutations at the amino acid level, because there aren't very many mutations relative to the number length of the nucleotide sequence for this gene, and I'd like to narrow the scope of the problem a little bit. So, I am also filtering out non-coding variants.

This leaves us with 175 / 217 variants.

`TODO:` __I should definitely search the literature for better/other formats__

`TODO:` __Find some more data__ 
>It would be really interesting to see if I could build a nucleotide level predictor, but also more data would really help make even the an amino acid model more robust (because right now we _really_ don't have enough data points to do anything serious, and this analysis is more for fun than anything else.

In [8]:
# select single nucleotide variants in the protein coding region
relevant_cols = ["variant", "clinical_significance"]
protein_snvs = clinvar[relevant_cols][~clinvar["is_indel"] & clinvar["is_in_protein"]]

In [9]:
# Process Protein Variant
get_protein_variant = lambda x: x.split("(p.")[1].split(")")[0] if is_protein_variant(x) else None
protein_snvs["protein_variant"] = protein_snvs["variant"].apply(get_protein_variant)

get_pos = lambda x: re.findall("\d+", x)[0]
protein_snvs["pos"] = protein_snvs["protein_variant"].apply(get_pos)

is_synonymous = lambda x: re.split("\d+", x)[1] == '='
protein_snvs["is_synonymous"] = protein_snvs["protein_variant"].apply(is_synonymous)

get_ref = lambda x: re.split("\d+", x)[0]
protein_snvs["ref"] = protein_snvs["protein_variant"].apply(get_ref)

get_alt = lambda x: re.split("\d+", x)[0] if re.split("\d+", x)[1] == '=' else re.split("\d+", x)[1]
protein_snvs["alt"] = protein_snvs["protein_variant"].apply(get_alt)

protein_snvs["is_missense"] = protein_snvs["alt"] == "Ter"

In [10]:
protein_snvs.head()

,variant,clinical_significance,protein_variant,pos,is_synonymous,ref,alt,is_missense
8,c.2976C>T (p.Asp992=),Benign,Asp992=,992,True,Asp,Asp,False
9,c.2974G>C (p.Asp992His),Uncertain significance,Asp992His,992,False,Asp,His,False
10,c.2974G>T (p.Asp992Tyr),Pathogenic,Asp992Tyr,992,False,Asp,Tyr,False
11,c.2968G>A (p.Val990Ile),Uncertain significance,Val990Ile,990,False,Val,Ile,False
12,c.2896G>A (p.Val966Met),Uncertain significance,Val966Met,966,False,Val,Met,False


## Categorize Variants as Pathogenic or Benign


In [11]:
protein_snvs["clinical_significance"].value_counts()

Uncertain significance                          60
Pathogenic                                      52
Likely benign                                   21
Likely pathogenic                               18
Benign/Likely benign                            11
Conflicting interpretations of pathogenicity     8
Benign                                           4
Pathogenic/Likely pathogenic                     2
Name: clinical_significance, dtype: int64

Variants seem to fall along a spectrum from benign to pathogenic, with some being uncertain.

__* Not sure how these are classified. Might be interesting for me to read more about this.__

For now, it seems reasonable to filter out variants with 'Uncertain significance' or 'Conflicting interpretations of pathogenicity', and put pathogenic/likely pathogenic variants in one group and benign/likely benign variants in another.

This leaves us with 108 / 176 protein coding SNVs.

In [12]:
is_uncertain = lambda x: x in {"Conflicting interpretations of pathogenicity", "Uncertain significance"}
protein_snvs["is_uncertain"] = protein_snvs["clinical_significance"].apply(is_uncertain)

is_pathogenic = lambda x: x in { "Pathogenic", "Likely pathogenic", "Pathogenic/Likely pathogenic"}
protein_snvs["is_pathogenic"] = protein_snvs["clinical_significance"].apply(is_pathogenic)

In [19]:
relevant_snv_cols = ['variant', 'pos', 'ref', 'alt', 'is_pathogenic', 'is_synonymous']
filtered_snvs = protein_snvs[~protein_snvs["is_uncertain"] & ~protein_snvs["is_missense"]][relevant_snv_cols]

In [20]:
filtered_snvs.head()

,variant,pos,ref,alt,is_pathogenic,is_synonymous
8,c.2976C>T (p.Asp992=),992,Asp,Asp,False,True
10,c.2974G>T (p.Asp992Tyr),992,Asp,Tyr,True,False
13,c.2886C>T (p.Pro962=),962,Pro,Pro,False,True
15,c.2864C>A (p.Ala955Asp),955,Ala,Asp,True,False
16,c.2859C>T (p.Ala953=),953,Ala,Ala,False,True


In [21]:
filtered_snvs["is_pathogenic"].value_counts()

True     71
False    36
Name: is_pathogenic, dtype: int64

#### Note:
There are twice as many pathogenic as non-pathogenic variants, something that should be taken into account when developing/testing our classifier.

## Exploratory Data Analysis

In [31]:
filtered_snvs.groupby(["is_pathogenic", "is_synonymous"]).count()

variant  pos  ref  alt
is_pathogenic is_synonymous                        
False         False                1    1    1    1
              True                35   35   35   35
True          False               71   71   71   71

According to this dataset, if a mutation in ATP1A3 is synonymous, it is not pathogenic.

If it is nonsynonymous, it is pathogenic the vast majority of the time.

Perhaps we don't need machine learning to build a classifier of this...